<a href="https://colab.research.google.com/github/harikrishnareddymallavarapu/schoolofai/blob/master/Session5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing the required libraries

In [ ]:
!pip install pytreebank
!pip install googletrans==3.1.0a0

### Loading the Stanford Sentiment Dataset from pytreebank

In [ ]:
import pytreebank
import pandas as pd
# dataset = pytreebank.load_sst("/path/to/sentiment/")
train_data = pytreebank.import_tree_corpus("/path/to/sentiment/train.txt")

FileNotFoundError: ignored

### Loading train, test and validation datasets into different dataframes. 

In [ ]:
sentenceData_train = []
sentenceData_val = []
sentenceData_test = []

for sentenceNode in dataset["train"]:
   sentenceData_train.append(sentenceNode.to_labeled_lines()[0])

for sentenceNode in dataset["test"]:
   sentenceData_test.append(sentenceNode.to_labeled_lines()[0])

for sentenceNode in dataset["dev"]:
   sentenceData_val.append(sentenceNode.to_labeled_lines()[0])

NameError: ignored

Converting the datasets to dataframes for next level processings

In [ ]:
sentenceData_train = pd.DataFrame(sentenceData_train, columns=['labels','sentence'])
sentenceData_train.head()

sentenceData_test = pd.DataFrame(sentenceData_test, columns=['labels','sentence'])
sentenceData_test.head()

sentenceData_val = pd.DataFrame(sentenceData_val, columns=['labels','sentence'])
sentenceData_val.head()


In [ ]:
sentenceData_train.describe(), sentenceData_train.shape,sentenceData_test.describe(), sentenceData_test.shape,sentenceData_val.describe(), sentenceData_val.shape

### Using Google Translate to augment a random number of data points in the dataset

In [ ]:
import random
import googletrans
from googletrans import Translator

translator = Translator()
# sentence = ['']

# available_langs = list(googletrans.LANGUAGES.keys()) 
# trans_lang = random.choice(available_langs) 
# print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")
# print(translator.translate("This is so cool"))

# translations = translator.translate(["This is cool"], dest=trans_lang) 
# t_text = [t.text for t in translations]
# print(t_text)

# translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
# en_text = [t.text for t in translations_en_random]
# print(en_text)
from tqdm import tqdm

def translateData(sentenceList):
  available_langs = list(googletrans.LANGUAGES.keys())
  transList = []
  for sentence in tqdm(sentenceList):
  
    trans_lang = random.choice(available_langs)
    translations = translator.translate(sentence, dest=trans_lang) 
    
    translations_en_random = translator.translate(translations.text, src=trans_lang, dest='en') 

    transList.append(str(translations_en_random.text))
  return transList




Random Swap function that randomly swaps words in a list, this function only swaps the position of the words

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    sentence = list(sentence)
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ''.join(sentence)

Random Deletion function that deletes words in a sentenc based on defined probability

In [ ]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

Generating a random sample of 5000 augmented sentences using google translator

In [ ]:
k = random.sample(range(0, sentenceData_train.shape[0]), 5000)

text = translateData(sentenceData_train.sentence[k])
#creating augmented data
transSentences = pd.DataFrame(list(zip(sentenceData_train.labels[k], text)),
               columns =['labels', 'sentence'])
transSentences.head()
# sentenceData = sentenceData.append(transSentences)
# sentenceData.shape[0]

 49%|████▉     | 2466/5000 [29:55<31:44,  1.33it/s]

In [ ]:
k = random.sample(range(0, sentenceData_train.shape[0]), 3000)
for i in k:
  sentenceData_train.sentence.loc[i] = random_swap(sentenceData_train.sentence[i],5)

In [ ]:
sentenceData_train = sentenceData_train.append(transSentences)
sentenceData_train.tail()

In [ ]:
sentenceData_train.to_excel("stanfordData2.xlsx", index=False)
sentenceData_train.shape

In [ ]:
# import pandas as pd
sentenceData = pd.read_excel("stanfordData2.xlsx")

In [ ]:
sentenceData = sentenceData.reset_index()

In [ ]:
sentenceData.describe()

,index,labels
count,21855.000000,21855.000000
mean,10927.000000,2.055548
std,6309.139402,1.288142
min,0.000000,0.000000
25%,5463.500000,1.000000
50%,10927.000000,2.000000
75%,16390.500000,3.000000
max,21854.000000,4.000000


In [ ]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [ ]:
fields = [('sentence', sentence),('labels',Label)]

In [ ]:
example = [data.Example.fromlist([sentenceData.sentence.iloc[i],sentenceData.labels.iloc[i]], fields) for i in range(sentenceData.shape[0])] 

In [ ]:
example2 = [data.Example.fromlist([sentenceData_val.sentence.iloc[i],sentenceData_val.labels.iloc[i]], fields) for i in range(sentenceData_val.shape[0])] 

In [ ]:
example3 = [data.Example.fromlist([sentenceData_test.sentence.iloc[i],sentenceData_test.labels.iloc[i]], fields) for i in range(sentenceData_test.shape[0])] 

In [ ]:
train = data.Dataset(example, fields)
# val = data.Dataset(example2, fields)
# test = data.Dataset(example3, fields)

In [ ]:
(train, valid, test) = train.split(split_ratio=[0.70, 0.15, 0.15], random_state=random.seed(SEED))

In [ ]:
(len(train), len(valid), len(test))

(15298, 3279, 3278)

In [ ]:
sentence.build_vocab(train)
Label.build_vocab(train)

In [ ]:
vars(train.examples[10])

{'labels': 4,
 'sentence': ['A',
  'deftly',
  'entertaining',
  'film',
  ',',
  'smartly',
  'played',
  'and',
  'smartly',
  'directed',
  '.']}

In [ ]:
print('Size of input vocab : ', len(sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  18876
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 14481), (',', 12851), ('the', 10805), ('and', 7923), ('of', 7787), ('a', 7783), ('to', 5467), ('-', 4893), ('is', 4496), ("'s", 4486)]
Labels :  defaultdict(None, {3: 0, 1: 1, 2: 2, 4: 3, 0: 4})


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(sentence.vocab.stoi, tokens)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(18876, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 5,904,905 trainable parameters


In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.sentence   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  

        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
       
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# !pip install tqdm

In [ ]:
from tqdm import tqdm
N_EPOCHS = 200
best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

  0%|          | 1/200 [00:51<2:51:50, 51.81s/it]

	Train Loss: 1.576 | Train Acc: 28.44%
	 Val. Loss: 1.562 |  Val. Acc: 32.13% 



  1%|          | 2/200 [01:40<2:47:36, 50.79s/it]

	Train Loss: 1.530 | Train Acc: 37.00%
	 Val. Loss: 1.514 |  Val. Acc: 38.65% 



  2%|▏         | 3/200 [02:28<2:44:38, 50.14s/it]

...

  6%|▌         | 11/200 [08:58<2:33:59, 48.89s/it]

	Train Loss: 1.167 | Train Acc: 74.91%
	 Val. Loss: 1.339 |  Val. Acc: 56.64% 



  6%|▌         | 12/200 [09:46<2:32:53, 48.80s/it]

	Train Loss: 1.150 | Train Acc: 76.51%
	 Val. Loss: 1.330 |  Val. Acc: 57.40% 



KeyboardInterrupt: ignored

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

In [ ]:
test_loss, test_acc